In [1]:
import os
os.chdir("../")

from wbe_odm import odm
from wbe_odm.odm_mappers import excel_template_mapper, mcgill_mapper, csv_mapper, ledevoir_mapper

import json
import requests
import pandas as pd
import plotly.express as px
import unidecode

cases, mortality, recovered, testing, active, avaccine, dvaccine or cvaccine

In [2]:
PLOTLY_COLORS = px.colors.qualitative.Plotly

COLORS = {
    "Very High": "#c13525",
    "High": "#ff8652",
    "Medium": "#ffbb43",
    "Low": "#b6e9d1",
    "Very Low": "#6da06f",
}

DATA_FOLDER = "/Users/jeandavidt/OneDrive - Université Laval/COVID/Latest Data"
CSV_FOLDER = "/Users/jeandavidt/OneDrive - Université Laval/COVID/Latest Data/odm_csv"
QC_STATIC_DATA = os.path.join(DATA_FOLDER, "Ville de Quebec - All data - v1.1.xlsx")
QC_LAB_DATA = os.path.join(DATA_FOLDER, "CentrEau-COVID_Resultats_Quebec_final.xlsx")
QC_SHEET_NAME =  "QC Data Daily Samples (McGill)"

MTL_STATIC_DATA = os.path.join(DATA_FOLDER, "mcgill_static.xlsx")
MTL_LAB_DATA = os.path.join(DATA_FOLDER, "CentrEau-COVID_Resultats_Montreal_final.xlsx")
MTL_POLY_SHEET_NAME = "Mtl Data Daily Samples (Poly)"
MTL_MCGILL_SHEET_NAME = "Mtl Data Daily Samples (McGill)"


In [3]:
RELOAD = True
store = odm.Odm()
if RELOAD:
    qc_lab = mcgill_mapper.McGillMapper()
    mcgill_lab = mcgill_mapper.McGillMapper()
    poly_lab = mcgill_mapper.McGillMapper()
    ledevoir = ledevoir_mapper.LeDevoirMapper()

    qc_lab.read(QC_LAB_DATA, QC_STATIC_DATA, QC_SHEET_NAME, "frigon_lab")
    mcgill_lab.read(MTL_LAB_DATA, MTL_STATIC_DATA, MTL_MCGILL_SHEET_NAME, "frigon_lab")
    poly_lab.read(MTL_LAB_DATA, MTL_STATIC_DATA, MTL_POLY_SHEET_NAME, "dorner_lab")
    ledevoir.read()

    store.append_from(qc_lab)
    store.append_from(mcgill_lab)
    store.append_from(poly_lab)
    store.append_from(ledevoir)
    store.to_csv(CSV_FOLDER, "a")

from_csv = csv_mapper.CsvMapper()
from_csv.read(CSV_FOLDER)
store.append_from(from_csv)

In [4]:
store.sample

,sampleID,siteID,instrumentID,reporterID,dateTime,dateTimeStart,dateTimeEnd,type,collection,preTreatment,pooled,children,parent,sizeL,index,fieldSampleTempC,shippedOnIce,storageTempC,qualityFlag,notes
0,frigon_lab_spb_1_qc_01_cptp24h _rawww_1,,,city_qc,NaT,2021-01-14,2021-01-15,rawww,cptp24h,,False,NaN,,1.0,1,NaN,NaN,4.0,False,
1,frigon_lab_spb_1_qc_02_cptp24h _rawww_1,,,city_qc,NaT,2021-01-14,2021-01-15,rawww,cptp24h,,False,NaN,,1,1,NaN,NaN,4.0,False,
6,frigon_lab_spb_1_reference_1,,,nielsnicolai,NaT,NaT,NaT,,,,False,NaN,,1,1,NaN,NaN,4.0,False,
7,frigon_lab_spb_1_negative_1,,,nielsnicolai,NaT,NaT,NaT,,,,False,NaN,,1,1,NaN,NaN,4.0,False,
8,frigon_lab_spb_2_qc_01_cptp24h _rawww_1,,,city_qc,NaT,2021-01-17,2021-01-18,rawww,cptp24h,,False,NaN,,1,1,NaN,NaN,4.0,False,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,mtl_8.2_grb3_raw_2021-04-19_1,mtl_8.2,,city_mtl,2021-04-19,NaT,2021-04-19,raw,grb,,False,NaN,,1,1,NaN,NaN,4.0,False,
373,mtl_8.1_ps_filter_2021-04-19_1,mtl_8.1,,city_mtl,NaT,2021-04-16,2021-04-19,filter,cptp72h,,False,NaN,,1,1,NaN,NaN,4.0,False,
374,mtl_8.2_ps_filter_2021-04-19_1,mtl_8.2,,city_mtl,NaT,2021-04-16,2021-04-19,filter,cptp72h,,False,NaN,,1,1,NaN,NaN,4.0,False,
377,mtl_01_cptp24h_raw_2021-04-20_1,mtl_01,,city_mtl,NaT,2021-04-19,2021-04-20,raw,cptp24h,,False,NaN,,1,1,NaN,NaN,4.0,False,


In [15]:
def get_site_geoJSON(odm_data):
    def make_point_feature(row):
        props_to_add = ["siteID", "name", "description", "clean_type", "polygonID"]
        return {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [row["geoLong"], row["geoLat"]],
                },
            "properties": {
                k: row[k] for k in props_to_add
            }
        }
    sites = odm_data.site
    site_types = {
        "wwtpmuc": "Station de traitement des eaux usées municipale pour égouts combinés",
        "pstat": "Station de pompage",
        "ltcf": "Établissement de soins de longue durée",
        "airpln":"Avion",
        "corFcil": "Prison",
        "school": "École",
        "hosptl": "Hôpital",
        "shelter": "Refuge",
        "swgTrck": "Camion de vidange",
        "uCampus": "Campus universitaire",
        "mSwrPpl": "Collecteur d'égouts",
        "holdTnk": "Bassin de stockage",
        "retPond": "Bassin de rétention",
        "wwtpMuS": "Station de traitement des eaux usées municipales pour égouts sanitaires seulement",
        "wwtpInd": "Station de traitement des eaux usées industrielle",
        "lagoon": "Système de lagunage pour traitement des eaux usées",
        "septTnk": "Fosse septique.",
        "river": "Rivière",
        "lake": "Lac",
        "estuary": "Estuaire",
        "sea": "Mer",
        "ocean": "Océan",
    }
    sites = odm_data.site
    sites["clean_type"] = sites["type"].str.lower().map(site_types)
    sites["clean_type"]
    sites["features"] = sites.apply(lambda row: make_point_feature(row), axis=1)
    point_list = list(sites["features"])
    js = {
        "type": "FeatureCollection",
        "features": point_list
    }
    return js


def get_samples_for_site(site_id, odm_data):
    samples = odm_data.sample
    sample_filter1 = samples["siteID"].str.lower() == site_id.lower()
    sample_filter2 = samples["type"].str.lower().str.contains("type_to_plot")
    return samples.loc[sample_filter1 & sample_filter2]

def get_plot_measures_for_site(site_id, odm_data):
    samples = get_samples_for_site(site_id, odm_data)
    ww = odm_data.ww_measure
    ww_filter1 = ww["type"].str.lower().isin(["covn2", "npmmov"])
    ww_filter2 = ww["unit"].str.lower().contains("gc")
    return ww.loc[ww_filter1 & ww_filter2]


def get_midpoint_time(date1, date2):
    if pd.isna(date1) or pd.isna(date2):
        return pd.NaT
    return date1 + (date2 - date1)/2


def get_plot_datetime(samples):
    # grb -> "dateTime"
    # ps and cp -> if start and end are present: midpoint
    # ps and cp -> if only end is present: end
    samples["plotDate"] = pd.NaT
    grb_filt = samples["type"].str.contains("grb")
    s_filt = ~samples["dateTimeStart"].isna()
    e_filt = ~samples["dateTimeEnd"].isna()

    samples.loc[grb_filt, "plotDate"] = samples.loc[grb_filt, "dateTime"]
    samples.loc[s_filt & e_filt, "plotDate"] = samples.apply(
        lambda row: get_midpoint_time(row["datetimeStart"], row["dateTimeEnd"])
    )
    samples.loc[e_filt & ~s_filt, "plotDate"] = samples.loc[e_filt & ~s_filt, "dateTimeEnd"]
    return samples["plotDate"]


def get_latest_sample_of_kind(samples, kind):
    kinds = ["ps", "cp", "grb"]
    if kind not in kinds:
        raise ValueError("Invalid kind")
    filt = samples["type"].str.contains(kind)
    df = samples.loc[filt]
    if len(df) == 0:
        return None
    df["plotDate"] = get_plot_datetime(df)
    df = df.sort_values(by="plotDate")
    return df.iloc[-1, df.columns.get_loc("plotDate")]


def get_type_to_plot(site_id, odm_data):
    # the type to plot depends on:
    # 1) What is the latest type of sample for that site
    # 2) How many samples of that type there are
    possible_types = ["ps", "cp", "grb"]
    thresh_n = 7  # n. of new sample of a certain kind required to switch type to plot
    samples = get_samples_for_site(site_id, odm_data)
    last_dates = []
    n_samples = []
    for type_ in possible_types:
        samples_of_type = samples.loc[samples["type"].str.contains(type_)]
        n = len(samples_of_type)
        last_dates.append(get_latest_sample_of_kind(samples, type_))
        n_samples.append(n)
    return possible_types, n_samples, last_dates
    
def get_plot_data(site_id, odm_data, type_to_plot):
    pass    

    
get_type_to_plot("QC_01", store)



(['ps', 'cp', 'grb'], [0, 0, 0], [None, None, None])

In [ ]:
polygons = store.polygon
polygons['color'] = "grey"
for i in range(len(polygons)):
    polygons.loc[i,"color"] = COLORS[i%len(COLORS)]
for col in ["pop", "link"]:
    if col in polygons.columns:
        polygons.drop(columns=[col], inplace=True)
polys = store.get_geoJSON()
with open("polygons.geojson", "w") as f:
    f.write(json.dumps(polys))